In [51]:
"""
LICENSE MIT
2021
Guillaume Rozier
Website : http://www.covidtracker.fr
Mail : guillaume.rozier@telecomnancy.net

README:
This file contains scripts that download data from data.gouv.fr and then process it to build many graphes.
I'm currently cleaning the code, please ask me if something is not clear enough.

The charts are exported to 'charts/images/france'.
Data is download to/imported from 'data/france'.
Requirements: please see the imports below (use pip3 to install them).

"""

"\nLICENSE MIT\n2021\nGuillaume Rozier\nWebsite : http://www.covidtracker.fr\nMail : guillaume.rozier@telecomnancy.net\n\nREADME:\nThis file contains scripts that download data from data.gouv.fr and then process it to build many graphes.\nI'm currently cleaning the code, please ask me if something is not clear enough.\n\nThe charts are exported to 'charts/images/france'.\nData is download to/imported from 'data/france'.\nRequirements: please see the imports below (use pip3 to install them).\n\n"

In [59]:
import pandas as pd
import json
import france_data_management as data
import math
import numpy as np

show_charts = False
PATH_STATS = "../../data/france/stats/"
PATH = "../../"

In [229]:
data.download_data()





  0%|          | 0/8 [00:00<?, ?it/s]



 12%|█▎        | 1/8 [00:00<00:06,  1.02it/s]



 38%|███▊      | 3/8 [00:01<00:04,  1.21it/s]



 75%|███████▌  | 6/8 [00:02<00:01,  1.65it/s]



28it [00:11,  1.80it/s]                      



36it [00:12,  2.51it/s]

In [56]:
df_regions_meta = pd.read_csv(PATH+"data/france/population_grandes_regions.csv")

In [57]:
#data.download_data_obepine()
#df_obepine = data.import_data_obepine()
#df_obepine_france = df_obepine.groupby("Date").mean().reset_index()

In [252]:
df_adm_hosp_clage = import_data_hosp_clage()
#df_adm_hosp_clage["jour"] = pd.to_datetime(df_adm_hosp_clage.Semaine+"-6", format="%Y-S%U-%w").fillna(0)
df_adm_hosp_clage_france = df_adm_hosp_clage.groupby(["jour", "cl_age90"]).sum().fillna(0).reset_index()
df_adm_hosp_clage

,reg,Semaine,cl_age90,NewAdmHospit,jour,regionName,regionPopulation,code
0,1,2020-S09,0,0,2020-03-01,Guadeloupe,395700,1
1,1,2020-S09,9,0,2020-03-01,Guadeloupe,395700,1
2,1,2020-S09,19,0,2020-03-01,Guadeloupe,395700,1
3,1,2020-S09,29,0,2020-03-01,Guadeloupe,395700,1
4,1,2020-S09,39,0,2020-03-01,Guadeloupe,395700,1
...,...,...,...,...,...,...,...,...
22963,94,2022-S19,59,0,2022-05-08,Corse,339178,94
22964,94,2022-S19,69,1,2022-05-08,Corse,339178,94
22965,94,2022-S19,79,3,2022-05-08,Corse,339178,94
22966,94,2022-S19,89,8,2022-05-08,Corse,339178,94


In [65]:
df, df_confirmed, dates, df_new, df_tests, df_deconf, df_sursaud, df_incid, df_tests_viros = data.import_data()




  0%|          | 0/8 [00:00<?, ?it/s]


 38%|███▊      | 3/8 [00:19<00:32,  6.60s/it]


 75%|███████▌  | 6/8 [00:21<00:09,  4.81s/it]


21it [00:28,  3.50s/it]                      


28it [04:34, 13.02s/it]


36it [04:35,  9.15s/it]

In [66]:
df_new_france = data.import_data_new().groupby("jour").sum().reset_index()

In [67]:
df_vue_ensemble = data.import_data_vue_ensemble()

In [152]:
def download_and_import_data_niveaux_scolaires_fra():
    df = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/f40a7dc4-31ce-4a38-b7b1-3fe6720606c7", sep=";")
    df["jour"] = df["sg"].apply(lambda x: x[-10:])
    df["jour"] = pd.to_datetime(df["jour"])
    df["age_18ans"] = df["cl_age18"].astype(str)
    df = df.sort_values(by="jour")
    return df.replace(",", ".", regex=True)

def download_and_import_data_niveaux_scolaires_reg():
    df = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/e8578f84-9ac7-43f3-9600-14bc8d4eeb1e", sep=";")
    df["jour"] = df["sg"].apply(lambda x: x[-10:])
    df["jour"] = pd.to_datetime(df["jour"])
    df = df.sort_values(by="jour")
    df_regions = pd.read_csv(PATH + 'data/france/departments_regions_france_2016.csv', sep=",")
    df = df.merge(df_regions, left_on="reg", right_on="regionCode")
    df["age_18ans"] = df["cl_age18"].astype(str)
    return df.replace(",", ".", regex=True)

def download_and_import_data_niveaux_scolaires_dep():
    df = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/356bde9d-18b6-4d8b-b8bb-150cba5fba74", sep=";")
    df["jour"] = df["sg"].apply(lambda x: x[-10:])
    df["jour"] = pd.to_datetime(df["jour"])
    df = df.sort_values(by="jour")
    df_regions = pd.read_csv(PATH + 'data/france/departments_regions_france_2016.csv', sep=",")
    df = df.merge(df_regions, left_on="dep", right_on="departmentCode")
    df["age_18ans"] = df["cl_age18"].astype(str)
    return df.replace(",", ".", regex=True)

In [153]:
df_education = data.import_data_education()

df_niveaux_scolaires_fra = data.download_and_import_data_niveaux_scolaires_fra()
df_niveaux_scolaires_reg = data.download_and_import_data_niveaux_scolaires_reg()
df_niveaux_scolaires_dep = data.download_and_import_data_niveaux_scolaires_dep()

In [107]:
#df_vacsi_a = data.import_data_vacsi_a_fra()
#df_vacsi_a_reg = data.import_data_vacsi_a_reg()
#df_vacsi_a_dep = data.import_data_vacsi_a_dep()

df_vacsi = data.import_data_vacsi_fra() #df_vacsi_a.groupby("jour").sum().reset_index()
df_vacsi_reg = data.import_data_vacsi_reg() #df_vacsi_a_reg.groupby(["jour", "reg"]).sum().reset_index()
df_vacsi_reg = df_vacsi_reg.merge(df_regions_meta, left_on="reg", right_on="code").rename({"n_tot_dose1": "n_cum_dose1"}, axis=1)

df_vacsi_dep = data.import_data_vacsi_dep().rename({"n_tot_dose1": "n_cum_dose1"}, axis=1)
#df_vacsi_a_dep.groupby(["jour", "dep"]).sum().reset_index().rename({"n_tot_dose1": "n_cum_dose1"}, axis=1)

In [111]:
df_metro = data.import_data_metropoles()
df_metro["jour"] = df_metro["sg"].map(lambda x: x[11:])

df_metro_65 = df_metro[df_metro["cl_age65"] == 65]
df_metro_0 = df_metro[df_metro["cl_age65"] == 0]
metropoles = list(dict.fromkeys(list(df_metro['Metropole'].dropna().values))) 

In [209]:
df_tests_viros_enrichi = data.import_data_tests_viros()
df_tests_viros_enrichi = df_tests_viros_enrichi.drop("regionName_y", axis=1).rename({"regionName_x": "regionName"}, axis=1)

In [113]:
df_incid_clage = df_incid.copy()

df_incid_fra_clage = data.import_data_tests_sexe()
df_incid_fra = df_incid_fra_clage[df_incid_fra_clage["cl_age90"]==0]
df_france = df.groupby(["jour"]).sum().reset_index()
df_incid = df_incid[df_incid.cl_age90 == 0]

df_sursaud_france = df_sursaud.groupby(["date_de_passage"]).sum().reset_index()
df_sursaud_regions = df_sursaud.groupby(["date_de_passage", "regionName"]).sum().reset_index()

#df_new_france = df_new.groupby(["jour"]).sum().reset_index()
df_new_regions = df_new.groupby(["jour", "regionName"]).sum().reset_index()

In [114]:
df_incid_clage_regions = df_incid_clage.groupby(["regionName", "jour", "cl_age90"]).sum().reset_index()

In [210]:
df_tests_viros_regions = df_tests_viros_enrichi.groupby(["regionName", "jour", "cl_age90"]).sum().reset_index()
df_tests_viros_france = df_tests_viros_enrichi.groupby(["jour", "cl_age90"]).sum().reset_index()

In [116]:
df_hosp_clage = data.import_data_hosp_clage()
df_hosp_clage_france = df_hosp_clage.groupby(["jour", "cl_age90"]).sum().reset_index()
df_hosp_clage_regions = df_hosp_clage.groupby(["regionName", "jour", "cl_age90"]).sum().reset_index()

In [117]:
departements = list(dict.fromkeys(list(df_incid['dep'].values)))
regions = list(dict.fromkeys(list(df_incid['regionName'].dropna().values))) 
clage_list = list(dict.fromkeys(list(df_incid_fra_clage['cl_age90'].dropna().values))) 

df_regions = df.groupby(["jour", "regionName"]).sum().reset_index()
df_incid_regions = df_incid.groupby(["jour", "regionName"]).sum().reset_index()


zone_a = ["zone_a", "01", "03", "07", "15", "16", "17", "19", "21", "23", "24", "25", "26", "33", "38", "39", "40", "42", "43", "47", "58", "63", "64", "69", "70", "71", "73", "74", "79", "86", "90"]
zone_b = ["zone_b", "02", "04", "05", "06", "08", "10", "13", "14", "18", "22", "27", "28", "29", "35", "36", "37", "41", "44", "45", "49", "50", "51", "52", "53", "54", "55", "56", "57", "59", "60", "61", "62", "67", "68", "72", "76", "80", "83", "84", "85", "88"]
zone_c = ["zone_c", "09", "11", "12", "30", "31", "32", "34", "46", "48", "65", "66", "75", "77", "78", "81", "82", "91", "92", "93", "94", "95"]

confines_mars_2021 = ["confines_mars_2021", "02", "06", "27", "59", "60", "62", "75", "76", "77", "78", "80", "91", "92", "93", "94", "95"]

In [119]:
df_opendata_indicateurs = data.download_and_import_opendata_indicateurs()

In [161]:
def generate_data(data_incid=pd.DataFrame(), data_hosp=pd.DataFrame(), data_sursaud=pd.DataFrame(), data_new=pd.DataFrame(), data_vue_ensemble=pd.DataFrame(), data_metropole=pd.DataFrame(), data_vacsi=pd.DataFrame(), data_obepine=pd.DataFrame(), data_opendata_indicateurs=pd.DataFrame(), mode="", export_jour=False, taux_croissance=False):## Incidence
        
    dict_data = {}
    
    if export_jour:
        dict_data["jour_incid"] = list(data_incid.jour)
        dict_data["jour_hosp"] = list(data_hosp.jour)
        dict_data["jour_new"] = list(data_new.jour)
        dict_data["jour_sursaud"] = list(data_sursaud.date_de_passage)
        dict_data["jour_metropoles"] = list(data_metropole.jour.unique())
        dict_data["jour_vacsi"] = list(data_vacsi.jour)
        dict_data["jour_obepine"] = "" #list(data_obepine.Date)
        
    if (taux_croissance) and (len(data_incid)>0):
        cas = data_incid["P"].fillna(0)
        taux_croissance_cas = ((cas-cas.shift(7))/cas.shift(7).replace(0, None)).fillna(0) * 100
        taux_croissance_cas[taux_croissance_cas>200]=200
        taux_croissance_cas[taux_croissance_cas<-200]=-200
        
        cas_rolling = data_incid["P"].rolling(window=7, center=True).mean().fillna(0)
        taux_croissance_cas_rolling = ((cas_rolling-cas_rolling.shift(7))/cas_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_cas"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_cas, 1))}
        dict_data["croissance_cas_rolling7"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_cas_rolling, 1))}
        
        tests = data_incid["T"].rolling(window=7).mean().fillna(0)
        taux_croissance_tests= ((tests-tests.shift(7))/tests.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_tests"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_tests, 1))}
        dict_data["croissance_tests_rolling7"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_tests.rolling(window=7, center=True).mean().fillna(0), 1))}
        
    if (taux_croissance) and (len(data_hosp)>0):
        hospitalisations = data_hosp.hosp.fillna(0)
        taux_croissance_hospitalisations = ((hospitalisations-hospitalisations.shift(7))/hospitalisations.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_hospitalisations"] = {"jour_nom": "jour_hosp", "valeur": list(round(taux_croissance_hospitalisations, 1))}
        dict_data["croissance_hospitalisations_rolling7"] = {"jour_nom": "jour_hosp", "valeur": list(round(taux_croissance_hospitalisations.rolling(window=7, center=True).mean().fillna(0), 1))}
        
        reanimations = data_hosp.rea.fillna(0)
        taux_croissance_reanimations = ((reanimations-reanimations.shift(7))/reanimations.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(round(taux_croissance_reanimations, 1))}
        dict_data["croissance_reanimations_rolling7"] = {"jour_nom": "jour_hosp", "valeur": list(round(taux_croissance_reanimations.rolling(window=7, center=True).mean().fillna(0), 1))}
    
    if (taux_croissance) and (len(data_new)>0):
        hospitalisations = data_new.incid_hosp.fillna(0)
        taux_croissance_hospitalisations = ((hospitalisations-hospitalisations.shift(7))/hospitalisations.shift(7).replace(0, None)).fillna(0) * 100
        
        hospitalisations_rolling = data_new.incid_hosp.fillna(0).rolling(window=7, center=True).mean()
        taux_croissance_hospitalisations_rolling = ((hospitalisations_rolling-hospitalisations_rolling.shift(7))/hospitalisations_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_incid_hospitalisations"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_hospitalisations, 1))}
        dict_data["croissance_incid_hospitalisations_rolling7"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_hospitalisations_rolling, 1))}
        
        reanimations = data_new.incid_rea.fillna(0)
        taux_croissance_reanimations = ((reanimations-reanimations.shift(7))/reanimations.shift(7).replace(0, None)).fillna(0) * 100
        
        reanimations_rolling = data_new.incid_rea.fillna(0).rolling(window=7, center=True).mean()
        taux_croissance_reanimations_rolling = ((reanimations_rolling-reanimations_rolling.shift(7))/reanimations_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_incid_reanimations"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_reanimations, 1))}
        dict_data["croissance_incid_reanimations_rolling7"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_reanimations_rolling, 1))}
    
    if (taux_croissance)and len(data_opendata_indicateurs):
        dict_data["jour_spf_opendata"] = list(data_opendata_indicateurs[["date", "conf_j1"]].dropna().date)
        
        data_opendata_indicateurs["conf_j1_corrige"] = data_opendata_indicateurs["conf"].diff()
        data_opendata_indicateurs.loc[data_opendata_indicateurs["date"]=="2021-05-20", "conf_j1_corrige"] = 15000
        
        #cas_spf_opendata = data_opendata_indicateurs["conf_j1_corrige"].fillna(0)
        cas_spf_opendata = data_opendata_indicateurs["conf_j1"].dropna()
            
        taux_croissance_cas_spf_opendata = ((cas_spf_opendata-cas_spf_opendata.shift(7))/cas_spf_opendata.shift(7).replace(0, None)).fillna(0) * 100
        cas_spf_opendata_rolling = cas_spf_opendata.rolling(window=7, center=True).mean().fillna(0)
        taux_croissance_cas_spf_opendata_rolling = ((cas_spf_opendata_rolling-cas_spf_opendata_rolling.shift(7))/cas_spf_opendata_rolling.shift(7).replace(0, None)).fillna(0) * 100
        
        dict_data["croissance_cas_spf_opendata"] = {"jour_nom": "jour_spf_opendata", "valeur": list(round(taux_croissance_cas_spf_opendata, 1))}
        dict_data["croissance_cas_spf_opendata_rolling7"] = {"jour_nom": "jour_spf_opendata", "valeur": list(round(taux_croissance_cas_spf_opendata_rolling, 1))}
        
        cas_spf_opendata_corrige = data_opendata_indicateurs[["date", "conf_j1_corrige"]]
        cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-05-02", "conf_j1_corrige"] = cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-04-25", "conf_j1_corrige"].values[0] * 0.7
        cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-05-09", "conf_j1_corrige"] = cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-05-02", "conf_j1_corrige"].values[0] * 0.7
        cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-05-14", "conf_j1_corrige"] = 0.7 * cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-05-07"]["conf_j1_corrige"].values[0]
        cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-05-25", "conf_j1_corrige"] = 0.7 * cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-05-18"]["conf_j1_corrige"].values[0]
        cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-07-15", "conf_j1_corrige"] = 1.8 * cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-07-08"]["conf_j1_corrige"].values[0]
        cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-11-02", "conf_j1_corrige"] = 1.1 * cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-10-26"]["conf_j1_corrige"].values[0]
        cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-11-12", "conf_j1_corrige"] = 1.3 * cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-11-05"]["conf_j1_corrige"].values[0]
        cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-12-26", "conf_j1_corrige"] = 1.5 * cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-12-19"]["conf_j1_corrige"].values[0]
        cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-01-01", "conf_j1_corrige"] = 2 * cas_spf_opendata_corrige.loc[cas_spf_opendata_corrige.date == "2021-12-25"]["conf_j1_corrige"].values[0]
        cas_spf_opendata_corrige = cas_spf_opendata_corrige["conf_j1_corrige"].rolling(window=7).mean().fillna(0)
        
        dict_data["cas_spf_opendata_rolling_corrige"] = {"jour_nom": "jour_spf_opendata", "valeur": list(round(cas_spf_opendata_corrige, 1))}

    if(len(data_vacsi)>0):
        n_cum_dose1 = data_vacsi["n_cum_dose1"].fillna(0)
        dict_data["n_cum_dose1"] = {"jour_nom": "jour_vacsi", "valeur": list(n_cum_dose1)}
        
        n_dose1 = data_vacsi["n_dose1"].rolling(window=7).mean().fillna(0)
        dict_data["n_dose1"] = {"jour_nom": "jour_vacsi", "valeur": list(round(n_dose1, 1))}
    
    if len(data_vue_ensemble)>0:
        dict_data["jour_ehpad"] = list(data_vue_ensemble.date)
        deces_ehpad = data_vue_ensemble["total_deces_ehpad"].diff().rolling(window=7).mean().fillna(0)
        dict_data["deces_ehpad"] = {"jour_nom": "jour_ehpad", "valeur": list(round(deces_ehpad,2))}
        
        cas_spf = data_vue_ensemble.total_cas_confirmes.diff().fillna(0)
        cas_spf[cas_spf<0] = 0
        cas_spf = cas_spf.replace(to_replace=0, method='ffill')
        cas_spf_rolling = cas_spf.rolling(window=7).mean().fillna(0)
        dict_data["cas_spf"] = {"jour_nom": "jour_ehpad", "valeur": list(round(cas_spf_rolling, 2))}
        dict_data["cas_spf_brut"] = {"jour_nom": "jour_ehpad", "valeur": list(round(cas_spf, 2))}
        
    if len(data_opendata_indicateurs):
        dict_data["jour_spf_opendata"] = list(data_opendata_indicateurs[["date", "conf_j1"]].dropna().date)
        #cas_spf_opendata = data_opendata_indicateurs["conf_j1_corrige"].fillna(0)
        cas_spf_opendata = data_opendata_indicateurs["conf_j1"].dropna()
        cas_spf_opendata_rolling = cas_spf_opendata.rolling(window=7).mean().fillna(0)
        
        dict_data["cas_spf_opendata"] = {"jour_nom": "jour_spf_opendata", "valeur": list(cas_spf_opendata)}
        dict_data["cas_spf_opendata_rolling"] = {"jour_nom": "jour_spf_opendata", "valeur": list(round(cas_spf_opendata_rolling, 2))}
        
        
    if len(data_obepine)>0:
        indicateur_obepine = data_obepine.Indicateur.fillna(0)
        
        dict_data["obepine"] = {"jour_nom": "jour_obepine", "jours":list(data_obepine.Date), "valeur": list(round(indicateur_obepine, 2))}
        
    if len(data_incid)>0:
        taux_incidence = data_incid["P"].rolling(window=7).sum().fillna(0) * 100000 / int(data_incid["pop"].values[0])
        dict_data["incidence"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_incidence, 1))}

        taux_positivite = (data_incid["P"] / data_incid["T"] * 100).rolling(window=7).mean().fillna(0)
        dict_data["taux_positivite"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_positivite, 2))}
        
        taux_positivite = (data_incid["P"].rolling(window=7).mean() / data_incid["T"].rolling(window=7).mean() * 100).fillna(0)
        dict_data["taux_positivite_rolling_before"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_positivite, 2))}
    
        cas = data_incid["P"].rolling(window=7).mean().fillna(0)
        dict_data["cas"] = {"jour_nom": "jour_incid", "valeur": list(round(cas, 1))}
        
        cas = data_incid["P"].fillna(0)
        dict_data["cas_brut"] = {"jour_nom": "jour_incid", "valeur": list(round(cas, 1))}
        
        cas_total = data_incid["P"].sum()
        dict_data["cas_total"] = {"jour_nom": "jour_incid", "valeur": int(cas_total)}
    
        tests = data_incid["T"].rolling(window=7).mean().fillna(0)
        dict_data["tests"] = {"jour_nom": "jour_incid", "valeur": list(round(tests, 1))}
        
        tests_total = data_incid["T"].sum()
        dict_data["tests_total"] = {"jour_nom": "jour_incid", "valeur": int(tests_total)}
        
    if (len(data_metropole)>0) & (mode=="metropoles"):
        taux_incidence = data_metropole["Ti"].fillna(0)
        dict_data["incidence"] = {"jour_nom": "jour_metropoles", "valeur": list(round(taux_incidence, 1))}
        
    if (taux_croissance) and (len(data_hosp)>0):
        hospitalisations_rolling = data_hosp.hosp.rolling(window=7).mean().fillna(0)
        croissance_hospitalisations = ((hospitalisations_rolling-hospitalisations_rolling.shift(7))/hospitalisations_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_hospitalisations"] = {"jour_nom": "jour_hosp", "valeur": list(round(croissance_hospitalisations, 1))}
        
        sc_rolling = data_hosp.rea.rolling(window=7).mean().fillna(0)
        croissance_sc = ((sc_rolling-sc_rolling.shift(7))/sc_rolling.shift(7).replace(0, None)).fillna(0) * 100
        dict_data["croissance_reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(round(croissance_sc, 1))}
        
    if len(data_hosp)>0:
        hospitalisations = data_hosp["hosp"].fillna(0)
        dict_data["hospitalisations"] = {"jour_nom": "jour_hosp", "valeur": list(hospitalisations)}

        reanimations = data_hosp.rea.fillna(0)
        dict_data["reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(reanimations)}
        
        saturation_rea = round(data_hosp["rea"]/data_hosp["LITS"].fillna(0)*100, 1)
        dict_data["saturation_reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(saturation_rea)}
    
    if len(data_new)>0:
        incid_hospitalisations = data_new.incid_hosp.rolling(window=7).mean().fillna(0)
        dict_data["incid_hospitalisations"] = {"jour_nom": "jour_new", "valeur": list(round(incid_hospitalisations, 1))}
        
        incid_hospitalisations_total = data_new.incid_hosp.sum()
        dict_data["incid_hospitalisations_total"] = {"jour_nom": "jour_new", "valeur": int(incid_hospitalisations_total)}

        incid_reanimations = data_new.incid_rea.rolling(window=7).mean().fillna(0)
        dict_data["incid_reanimations"] = {"jour_nom": "jour_new", "valeur": list(round(incid_reanimations, 1))}
        
        incid_reanimations_total = data_new.incid_rea.sum()
        dict_data["incid_reanimations_total"] = {"jour_nom": "jour_new", "valeur": int(incid_reanimations_total)}
    
    if len(data_sursaud)>0:
        nbre_acte_corona = data_sursaud.nbre_acte_corona.rolling(window=7).mean().fillna(0)
        dict_data["nbre_acte_corona"] = {"jour_nom": "jour_sursaud", "valeur": list(round(nbre_acte_corona, 1))}

        nbre_pass_corona = data_sursaud.nbre_pass_corona.rolling(window=7).mean().fillna(0)
        dict_data["nbre_pass_corona"] = {"jour_nom": "jour_sursaud", "valeur": list(round(nbre_pass_corona,  1))}
    
    if len(data_new)>0:
        deces_hospitaliers = data_new.incid_dc
        taux_croissance_deces_hospitaliers = ((deces_hospitaliers-deces_hospitaliers.shift(7))/deces_hospitaliers.shift(7).replace(0, None)).fillna(0) * 100
        
        deces_hospitaliers_rolling = data_new.incid_dc.rolling(window=7, center=False).mean().fillna(0)
        taux_croissance_deces_hospitaliers_rolling = ((deces_hospitaliers_rolling-deces_hospitaliers_rolling.shift(7))/deces_hospitaliers_rolling.shift(7).replace(0, None)).fillna(0) * 100
        
        dict_data["deces_hospitaliers"] = {"jour_nom": "jour_new", "valeur": list(round(deces_hospitaliers_rolling, 1))}
        dict_data["croissance_deces_hospitaliers"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_deces_hospitaliers, 1))}
        dict_data["croissance_deces_hospitaliers_rolling7"] = {"jour_nom": "jour_new", "valeur": list(round(taux_croissance_deces_hospitaliers_rolling, 1))}
        
        deces_hospitaliers_total = data_new.incid_dc.sum()
        dict_data["deces_hospitaliers_total"] = {"jour_nom": "jour_new", "valeur": int(deces_hospitaliers_total)}
    
    if len(data_incid)>0:
        population = data_incid["pop"].values[0]
        dict_data["population"] = population

    return dict_data

In [231]:
def generate_data_age(data_incid, data_hosp, data_adm_hosp_clage=pd.DataFrame(), export_jour=False):## Incidence
    clage_tranches = [["9", "19", "29", "39", "49", "59", "69", "79", "89", "90"], ["9", "19"], ["29", "39"], ["49", "59"], ["69", "79"], ["89", "90"]]
    clage_noms = ["tous", "19", "39", "59", "79", "90"]
    clage_noms_disp = ["Tous âges", "0 à 19 ans", "20 à 39 ans", "40 à 59 ans", "60 à 79 ans", "Plus de 80 ans"]
    
    dict_data = {}
    print(data_incid)
    for (idx, clage) in enumerate(clage_tranches):
        clage_nom = clage_noms[idx]
        
        data_incid_clage = data_incid[data_incid.cl_age90.isin(clage)].groupby("jour").sum().reset_index()

        dict_data[clage_nom] = {}
        
        taux_incidence = data_incid_clage["P"].rolling(window=7).sum().fillna(0) * 100000 / data_incid_clage["pop"].values[0]
        dict_data[clage_nom]["incidence"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_incidence,0))}

        taux_positivite = (data_incid_clage["P"] / data_incid_clage["T"] * 100).rolling(window=7).mean().fillna(0)
        dict_data[clage_nom]["taux_positivite"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_positivite,2))}

        cas = data_incid_clage["P"].rolling(window=7).mean().fillna(0)
        dict_data[clage_nom]["cas"] = {"jour_nom": "jour_incid", "valeur": list(round(cas, 1))}
        
        taux_croissance_cas = ((cas-cas.shift(7))/cas.shift(7).replace(0, None)).fillna(0) * 100
        taux_croissance_cas[taux_croissance_cas>200] = 200
        taux_croissance_cas[taux_croissance_cas<-200] = -200
        #taux_croissance_cas = taux_croissance_cas.rolling(window=7).mean().fillna(0)
        dict_data[clage_nom]["cas_croissance_hebdo"] = {"jour_nom": "jour_incid", "valeur": list(round(taux_croissance_cas, 1))}

        tests = data_incid_clage["T"].rolling(window=7).mean().fillna(0)
        dict_data[clage_nom]["tests"] = {"jour_nom": "jour_incid", "valeur": list(round(tests, 1))}
        
        population = data_incid_clage["pop"].values[0]
        dict_data[clage_nom]["population"] = population
        
        if (len(data_hosp)):  
            data_hosp_clage = data_hosp[data_hosp.cl_age90.isin(clage)].groupby("jour").sum().reset_index()
            hospitalisations = data_hosp_clage.hosp.fillna(0)
            dict_data[clage_nom]["hospitalisations"] = {"jour_nom": "jour_hosp", "valeur": list(hospitalisations)}

            reanimations = data_hosp_clage.rea.fillna(0)
            dict_data[clage_nom]["reanimations"] = {"jour_nom": "jour_hosp", "valeur": list(reanimations)}

            deces_hospitaliers = data_hosp_clage.dc.diff().rolling(window=7).mean().fillna(0)
            dict_data[clage_nom]["deces_hospitaliers"] = {"jour_nom": "jour_hosp", "valeur": list(round(deces_hospitaliers, 1))}
        
        if(len(data_adm_hosp_clage)):
            print(data_adm_hosp_clage)
            df_adm_hosp_clage_temp = data_adm_hosp_clage[data_adm_hosp_clage.cl_age90.isin(clage)].groupby("jour").sum().reset_index()
            adm_hospitalisations = df_adm_hosp_clage_temp["NewAdmHospit"]
            dict_data[clage_nom]["adm_hospitalisations"] = {"jour_nom": "jour_adm_hosp_clage", "valeur": list(round(adm_hospitalisations/7, 1))}
            
    if export_jour:
            dict_data["jour_incid"] = list(data_incid.jour.unique())
            dict_data["jour_hosp"] = list(data_hosp.jour.unique())
            dict_data["tranches"] = clage_tranches
            dict_data["tranches_noms"] = clage_noms
            dict_data["tranches_noms_affichage"] = clage_noms_disp
            
            if(len(data_adm_hosp_clage)):
                dict_data["jour_adm_hosp_clage"] = list(data_adm_hosp_clage.jour.dt.strftime('%Y-%m-%d').unique())

    return dict_data
 

In [167]:
def generate_data_niveaux_scolaires(data_incid_niveaux_scolaires, export_jour=False):
    dict_data={}
    clage_tranches = ["0", "2", "5", "10", "14", "17", "18"]
    clage_noms = ["tous_scol", "02_scol", "05_scol", "10_scol", "14_scol", "17_scol", "18_scol"]
    clage_noms_disp = ["Tous", "0 - 2 ans", "3 - 5 ans", "6 - 10 ans", "11 - 14 ans", "15 - 17 ans", "plus 18 ans"]
    
    for (idx, clage) in enumerate(clage_tranches):
        clage_nom = clage_noms[idx]
        data_incid_niveaux_scolaires_clage = data_incid_niveaux_scolaires[data_incid_niveaux_scolaires.age_18ans == clage].reset_index()

        dict_data[clage_nom] = {}

        taux_incidence = data_incid_niveaux_scolaires_clage["Ti"]
        taux_depistage = data_incid_niveaux_scolaires_clage["Td"]
        taux_positivite = data_incid_niveaux_scolaires_clage["Tp"]

        dict_data[clage_nom]["incidence"] = {"jour_nom": "jour_niveaux_scolaires", "valeur": round(taux_incidence.astype(float)).astype(int).tolist()}
        dict_data[clage_nom]["depistage"] = {"jour_nom": "jour_niveaux_scolaires", "valeur": round(taux_depistage.astype(float)).astype(int).tolist()}
        dict_data[clage_nom]["positivite"] = {"jour_nom": "jour_niveaux_scolaires", "valeur": round(taux_positivite.astype(float), 1).tolist()}
            
    if export_jour:
            dict_data["jour_niveaux_scolaires"] = list(data_incid_niveaux_scolaires.jour.unique().astype(str))
            dict_data["tranches_scolaires"] = clage_tranches
            dict_data["tranches_noms_scolaires"] = clage_noms
            dict_data["tranches_noms_affichage_scolaires"] = clage_noms_disp

    return dict_data
 

In [140]:
def generate_data_education(df_education):
    clage_noms = [2, 5, 10, 14, 17, 18]
    clage_disp = ["0 - 2 ans", "3 - 5 ans", "6 - 10 ans", "11 - 14 ans", "15 - 17 ans", "Plus 18 ans"]
    
    dict_data = {}
    dict_data["tranches_noms"] = clage_noms
    dict_data["tranches_noms_affichage"] = clage_disp
    
    for clage in clage_noms:
        dict_data[clage] = {}
        
        df_temp = df_education[df_education["age_18ans"]==clage]
        
        dict_data[clage]["Ti"] = df_temp["Ti"].fillna(0).tolist()
        dict_data[clage]["Tp"] = df_temp["Tp"].fillna(0).tolist()
        dict_data[clage]["Td"] = df_temp["Td"].fillna(0).tolist()
        
    return dict_data

In [124]:
def export_data(data, suffix=""):
    with open(PATH_STATS + 'dataexplorer{}.json'.format(suffix), 'w') as outfile:
        json.dump(data, outfile)

In [163]:
def dataexplorer():
    dict_data = {}
    
    regions = sorted(list(df_regions.regionName.unique()))
    departements = list(df.dep.unique())
    
    dict_data["regions"] = regions
    dict_data["metropoles"] = sorted(metropoles)
    dict_data["departements"] = departements
    dict_data["france"] = generate_data(df_incid_fra, df_france, df_sursaud_france, df_new_france, df_vue_ensemble, data_metropole=df_metro_0, data_vacsi=df_vacsi, data_obepine=pd.DataFrame(), data_opendata_indicateurs=df_opendata_indicateurs, mode="france", export_jour=True, taux_croissance=True)
    dict_data["metropole"] = generate_data(df_incid[df_incid["dep"].str.len()<=2].groupby(["jour"]).sum(), df[df["dep"].str.len()<=2].groupby(["jour"]).sum(), df_sursaud[df_sursaud["dep"].str.len()<=2].groupby(["date_de_passage"]).sum(), df_new[df_new["dep"].str.len()<=2].groupby(["jour"]).sum(), data_vacsi=df_vacsi_dep[df_new["dep"].str.len()<=2].groupby(["jour"]).sum())
    dict_data["drom_com"] = generate_data(df_incid[df_incid["dep"].str.len()>2].groupby(["jour"]).sum(), df[df["dep"].str.len()>2].groupby(["jour"]).sum(), df_sursaud[df_sursaud["dep"].str.len()>2].groupby(["date_de_passage"]).sum(), df_new[df_new["dep"].str.len()>2].groupby(["jour"]).sum(), data_vacsi=df_vacsi_dep[df_new["dep"].str.len()>2].groupby(["jour"]).sum())

    noms_departements={}
    
    for reg in regions:
        print(reg)
        data_hosp = df_regions[df_regions.regionName==reg]
        dict_data[reg] = generate_data(data_incid=df_incid_regions[df_incid_regions.regionName==reg], \
                                       data_hosp=data_hosp,\
                                       data_sursaud=df_sursaud_regions[df_sursaud_regions.regionName==reg],
                                       data_new=df_new_regions[df_new_regions.regionName==reg],
                                       data_vacsi=df_vacsi_reg[df_vacsi_reg.regionName==reg],\
                                       data_obepine=pd.DataFrame(), #df_obepine[df_obepine.regionName==reg]
                                      )
    print("Regions : done")
    
    for dep in departements:
        print(dep)
        df_incid_dep = df_incid[df_incid.dep==dep]
        df_dep = df[df.dep==dep]
        dict_data[dep] = generate_data(data_incid=df_incid_dep, 
                                       data_hosp=df_dep,
                                       data_sursaud=df_sursaud[df_sursaud.dep==dep],
                                       data_new=df_new[df_new.dep==dep],
                                       data_vacsi=df_vacsi_dep[df_vacsi_dep.dep==dep])
        noms_departements[dep] = df_dep["departmentName"].values[0]
    dict_data["departements_noms"] = noms_departements
    
    for zone in [zone_a, zone_b, zone_c]:
        df_incid_zone = df_incid[df_incid.dep.isin(zone)].groupby("jour").sum().reset_index()
        df_zone = df[df.dep.isin(zone)].groupby("jour").sum().reset_index()
        df_sursaud_zone = df_sursaud[df_sursaud.dep.isin(zone)].groupby("date_de_passage").sum().reset_index()
        df_new_zone = df_new[df_new.dep.isin(zone)].groupby("jour").sum().reset_index()
        df_vacsi_zone = df_vacsi_dep[df_vacsi_dep.dep.isin(zone)].groupby("jour").sum().reset_index()
        
        dict_data[zone[0]] = generate_data(df_incid_zone, df_zone, df_sursaud_zone, df_new_zone, data_vacsi=df_vacsi_zone)
    
    # Confinés mars 2021
    df_incid_zone = df_incid[df_incid.dep.isin(confines_mars_2021)].groupby("jour").sum().reset_index()
    df_zone = df[df.dep.isin(confines_mars_2021)].groupby("jour").sum().reset_index()
    df_sursaud_zone = df_sursaud[df_sursaud.dep.isin(confines_mars_2021)].groupby("date_de_passage").sum().reset_index()
    df_new_zone = df_new[df_new.dep.isin(confines_mars_2021)].groupby("jour").sum().reset_index()
    df_vacsi_zone = df_vacsi_dep[df_vacsi_dep.dep.isin(confines_mars_2021)].groupby("jour").sum().reset_index()
    
    dict_data["confines_mars_2021"] = generate_data(df_incid_zone, df_zone, df_sursaud_zone, df_new_zone, data_vacsi=df_vacsi_zone)
        
    for metropole in metropoles:
        print(metropole)
        dict_data[metropole] = generate_data(data_metropole=df_metro_0[df_metro_0.Metropole == metropole], mode="metropoles")
        
    dict_data["zones_vacances"] = ["zone_a", "zone_b", "zone_c"]
    
    export_data(dict_data, suffix="_compr")
    export_data(dict_data["france"], suffix="_compr_france")
    return dict_data

In [180]:
def dataexplorer_age():
    dict_data = {}
    regions_tests_viros = list(dict.fromkeys(list(df_tests_viros_enrichi['regionName'].dropna().values))) 
    departements_tests_viros = list(dict.fromkeys(list(df_tests_viros_enrichi['dep'].dropna().values))) 
    dict_data["regions"] = sorted(regions_tests_viros)
    dict_data["departements"] = sorted(departements_tests_viros)

    dict_data["france"] = generate_data_age(data_incid=df_tests_viros_france, data_hosp=df_hosp_clage_france, data_adm_hosp_clage=df_adm_hosp_clage_france, export_jour=True)

    for reg in regions_tests_viros:
        dict_data[reg] = generate_data_age(df_tests_viros_regions[df_tests_viros_regions.regionName == reg],\
                                           df_hosp_clage_regions[df_hosp_clage_regions.regionName == reg]) #data_adm_hosp_clage=data_adm_hosp_clage
    noms_departements={}
    for dep in departements_tests_viros:
        df_tests_viros_enrichi_temp = df_tests_viros_enrichi[df_tests_viros_enrichi.dep == dep]
        dict_data[dep] = generate_data_age(df_tests_viros_enrichi_temp,\
                                           pd.DataFrame())
        
        nom_dep = df_tests_viros_enrichi_temp["departmentName"].values[0]
        
        if(type(nom_dep) is float): #Pas de nom, nom_dep == NaN
            nom_dep = "--"
        
        noms_departements[dep] = nom_dep
        
    dict_data["departements_noms"] = noms_departements
    
    export_data(dict_data, suffix="_compr_age")
    return dict_data

In [127]:
def dataexplorer_education():
    dict_data = {}
    dict_data["france"] = generate_data_education(df_education=df_education)
    export_data(dict_data, suffix="_education")

In [128]:
def dataexplorer_niveaux_scolaires():
    df_niveaux_scolaires_dep_sorted = df_niveaux_scolaires_dep.sort_values(by="dep")
    df_niveaux_scolaires_reg_sorted = df_niveaux_scolaires_reg.sort_values(by="reg")
    
    dict_data = {}
    
    dict_data["departements_noms"] = list(df_niveaux_scolaires_dep_sorted["departmentName"].unique().astype(str))
    dict_data["departements"] = list(df_niveaux_scolaires_dep_sorted["dep"].unique().astype(str))
    
    dict_data["regions_noms"] = list(df_niveaux_scolaires_reg_sorted["regionName"].unique().astype(str))
    dict_data["regions"] = dict_data["regions_noms"] #list(df_niveaux_scolaires_reg_sorted["reg"].unique().astype(str))
    
    dict_data["france"] = generate_data_niveaux_scolaires(data_incid_niveaux_scolaires=df_niveaux_scolaires_fra, export_jour=True)
    
    for dep in dict_data["departements"]:
        data_incid_niveaux_scolaires_dep_temp = df_niveaux_scolaires_dep[df_niveaux_scolaires_dep["dep"] == dep]
        dict_data[dep] = generate_data_niveaux_scolaires(data_incid_niveaux_scolaires=data_incid_niveaux_scolaires_dep_temp, export_jour=False)
        
    for reg in dict_data["regions_noms"]:
        print(reg)
        data_incid_niveaux_scolaires_reg_temp = df_niveaux_scolaires_reg[df_niveaux_scolaires_reg["regionName"] == reg]
        dict_data[reg] = generate_data_niveaux_scolaires(data_incid_niveaux_scolaires=data_incid_niveaux_scolaires_reg_temp, export_jour=False)
    
    export_data(dict_data, suffix="_education")
    return dict_data

In [129]:
def data_nouveau_dashboard_france():
    data={}
    name="hospitalisations_par_age"
    
    df = df_hosp_clage_france[df_hosp_clage_france["jour"]==df_hosp_clage_france["jour"].max()]
    df = df[df["cl_age90"] != 0].sort_values(by="cl_age90")
    
    data["cl_age90"] = ["0-9 ans", "10-19 ans", "20-29 ans", "30-39 ans", "40-49 ans", "50-59 ans", "60-69 ans", "70-79 ans", "80-89 ans", "+ 90 ans"]
    data["hosp"] = df["hosp"].fillna(0).tolist()
    data["rea"] = df["rea"].fillna(0).tolist()
    
    with open(PATH_STATS + 'api/{}.json'.format(name), 'w') as outfile:
        json.dump(data, outfile)

In [131]:
data_nouveau_dashboard_france()

In [168]:
dict_ns = dataexplorer_niveaux_scolaires()

Series([], Name: Ti, dtype: float64)
0        0.0
1        1.0
2        1.0
3        1.0
4        1.0
       ...  
732    174.0
733    160.0
734    152.0
735    142.0
736    134.0
Name: Ti, Length: 737, dtype: float64
0        0.0
1        0.0
2        0.0
3        1.0
4        1.0
       ...  
732    217.0
733    194.0
734    180.0
735    166.0
736    153.0
Name: Ti, Length: 737, dtype: float64
0        0.0
1        1.0
2        1.0
3        1.0
4        1.0
       ...  
732    287.0
733    268.0
734    254.0
735    242.0
736    228.0
Name: Ti, Length: 737, dtype: float64
0        0.0
1        1.0
2        1.0
3        1.0
4        1.0
       ...  
732    271.0
733    260.0
734    254.0
735    245.0
736    233.0
Name: Ti, Length: 737, dtype: float64
0        0.0
1        1.0
2        1.0
3        1.0
4        1.0
       ...  
732    210.0
733    198.0
734    190.0
735    184.0
736    177.0
Name: Ti, Length: 737, dtype: float64
0        2.0
1        3.0
2        5.0
3        6.0
4     

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
732    260.0
733    242.0
734    236.0
735    206.0
736    188.0
Name: Ti, Length: 737, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
732    366.0
733    335.0
734    298.0
735    278.0
736    254.0
Name: Ti, Length: 737, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
732    223.0
733    227.0
734    219.0
735    190.0
736    215.0
Name: Ti, Length: 737, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
732    197.0
733    217.0
734    222.0
735    217.0
736    222.0
Name: Ti, Length: 737, dtype: float64
0        1.0
1        3.0
2        4.0
3        5.0
4        5.0
       ...  
732    378.0
733    347.0
734    321.0
735    305.0
736    292.0
Name: Ti, Length: 737, dtype: float64
Series([], Name: Ti, dtype: float64)
0        0.0
1        0.0
2        0.0
3        0.0
4     

Name: Ti, Length: 736, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
725    245.0
726    184.0
727    102.0
728     82.0
729     61.0
Name: Ti, Length: 730, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
732    267.0
733    244.0
734    255.0
735    267.0
736    255.0
Name: Ti, Length: 737, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
731    196.0
732    211.0
733    211.0
734    180.0
735    180.0
Name: Ti, Length: 736, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
732    134.0
733    179.0
734    201.0
735    246.0
736    246.0
Name: Ti, Length: 737, dtype: float64
0        3.0
1        4.0
2        5.0
3        5.0
4        5.0
       ...  
732    374.0
733    331.0
734    287.0
735    285.0
736    277.0
Name: Ti, Length: 737, dtype: float64
Series([], Name: Ti, dtype: float64)
0        0.0
1      

Series([], Name: Ti, dtype: float64)
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
731    116.0
732     33.0
733     50.0
734     50.0
735     83.0
Name: Ti, Length: 736, dtype: float64
Series([], Name: Ti, dtype: float64)
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
731    315.0
732    291.0
733    354.0
734    330.0
735    315.0
Name: Ti, Length: 736, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
731    288.0
732    261.0
733    288.0
734    288.0
735    279.0
Name: Ti, Length: 736, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
       ...  
730    311.0
731    287.0
732    251.0
733    215.0
734    179.0
Name: Ti, Length: 735, dtype: float64
Series([], Name: Ti, dtype: float64)
Series([], Name: Ti, dtype: float64)
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
726    79.0
727    53.0
728    79.0
729    79.0
730    5

Series([], Name: Ti, dtype: float64)
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
     ... 
93    0.0
94    0.0
95    0.0
96    0.0
97    0.0
Name: Ti, Length: 98, dtype: float64
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
110    0.0
111    0.0
112    0.0
113    0.0
114    0.0
Name: Ti, Length: 115, dtype: float64
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
327    0.0
328    0.0
329    0.0
330    0.0
331    0.0
Name: Ti, Length: 332, dtype: float64
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
383    0.0
384    0.0
385    0.0
386    0.0
387    0.0
Name: Ti, Length: 388, dtype: float64
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
373    0.0
374    0.0
375    0.0
376    0.0
377    0.0
Name: Ti, Length: 378, dtype: float64
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
724    43.0
725    86.0
726    86.0
727    86.0
728    86.0
Name: Ti, Length: 729, dtype: float64
Series([],

Name: Ti, Length: 737, dtype: float64
0       6.0
1      12.0
2      21.0
3      21.0
4      21.0
       ... 
732    45.0
733    49.0
734    48.0
735    43.0
736    35.0
Name: Ti, Length: 737, dtype: float64
Ile-de-France
Series([], Name: Ti, dtype: float64)
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
5891    89.0
5892    89.0
5893    89.0
5894    89.0
5895    89.0
Name: Ti, Length: 5896, dtype: float64
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
5891    92.0
5892    92.0
5893    92.0
5894    92.0
5895    92.0
Name: Ti, Length: 5896, dtype: float64
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
        ...  
5891    124.0
5892    124.0
5893    124.0
5894    124.0
5895    124.0
Name: Ti, Length: 5896, dtype: float64
0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
        ...  
5891    106.0
5892    106.0
5893    106.0
5894    106.0
5895    106.0
Name: Ti, Length: 5896, dtype:

0         0.0
1         0.0
2         0.0
3         0.0
4         1.0
        ...  
2943    351.0
2944    336.0
2945    336.0
2946    336.0
2947    336.0
Name: Ti, Length: 2948, dtype: float64
Nouvelle-Aquitaine
Series([], Name: Ti, dtype: float64)
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
        ...  
8839    103.0
8840    103.0
8841    103.0
8842    103.0
8843    103.0
Name: Ti, Length: 8844, dtype: float64
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
        ...  
8839    137.0
8840    137.0
8841    137.0
8842    137.0
8843    137.0
Name: Ti, Length: 8844, dtype: float64
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
        ...  
8839    167.0
8840    167.0
8841    167.0
8842    167.0
8843    167.0
Name: Ti, Length: 8844, dtype: float64
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
        ...  
8839    223.0
8840    223.0
8841    223.0
8842    223.0
8843    223.0
Name: Ti, Length: 

In [164]:
dict_dataexplorer = dataexplorer()

/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


Auvergne-Rhône-Alpes
Bourgogne-Franche-Comté
Bretagne
Centre
Corse
Grand Est
Guadeloupe
Guyane
Hauts-de-France
Ile-de-France
La Réunion
Martinique
Mayotte
Normandie
Nouvelle-Aquitaine
Occitanie
Pays de la Loire
Provence-Alpes-Côte d'Azur
Regions : done
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
21
22
23
24
25
26
27
28
29
2A
2B
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
971
972
973
974
976
Rouen
Nice
Metz
Grenoble
Lyon
Paris
Marseille
Dijon
Brest
Toulouse
Bordeaux
Montpellier
Rennes
Tours
Saint Etienne
Nantes
Orléans
Nancy
Lille
Clermont-Auvergne
Strasbourg
Toulon


In [250]:
dict_data = dataexplorer_age()

            jour  cl_age90        pop       P        T  regionCode  \
0     2020-05-13         9  7539692.0    13.0   1277.0      5209.0   
1     2020-05-13        19  8243417.0    28.0   1678.0      5231.0   
2     2020-05-13        29  7430487.0   103.0   4610.0      5315.0   
3     2020-05-13        39  8268700.4   106.0   5661.0      5234.0   
4     2020-05-13        49  8549692.0   101.0   5813.0      5315.0   
...          ...       ...        ...     ...      ...         ...   
7365  2022-05-19        59  8792674.7  3340.0  19805.0      5318.0   
7366  2022-05-19        69  8004082.7  2987.0  17767.0      5318.0   
7367  2022-05-19        79  5695639.1  2510.0  14276.0      5318.0   
7368  2022-05-19        89  3245280.6   928.0   7931.0      5318.0   
7369  2022-05-19        90   912501.7   262.0   2927.0      5318.0   

      regionPopulation    code  
0          536721585.0  5209.0  
1          537914658.0  5231.0  
2          545941658.0  5315.0  
3          538205349.0  523

KeyError: 'jour'

In [ ]:
def objectif_deconfinement():
    dict_json = {}
    n = 70
    
    ## HOSP
    struct = {"dates": [], "values": []}
    dict_json["hosp"] = struct
    dict_json["hosp"]["values"] = [float(round(x, 1)) for x in df_france["hosp"].values[-n:]]
    dict_json["hosp"]["dates"] = list(df_france["jour"].values[-n:])
    
    ## HOSP ADM
    struct = {"dates": [], "values": []}
    dict_json["adm_hosp"] = struct
    dict_json["adm_hosp"]["values"] = [float(round(x, 1))for x in df_new_france["incid_hosp"].rolling(window=7).mean().dropna(0).values[-n:]]
    dict_json["adm_hosp"]["dates"] = list(df_new_france["jour"].values[-n:])
    
    ## REA
    struct = {"dates": [], "values": []}
    dict_json["rea"] = struct
    dict_json["rea"]["values"] = [float(round(x, 1)) for x in df_france["rea"].values[-n:]]
    dict_json["rea"]["dates"] = list(df_france["jour"].values[-n:])
    
    ## REA ADM
    struct = {"dates": [], "values": []}
    dict_json["adm_rea"] = struct
    dict_json["adm_rea"]["values"] = [float(round(x, 1)) for x in df_new_france["incid_rea"].rolling(window=7).mean().dropna(0).values[-n:]]
    dict_json["adm_rea"]["dates"] = list(df_new_france["jour"].values[-n:])
    
    ## DC
    struct = {"dates": [], "values": []}
    dict_json["dc"] = struct
    dict_json["dc"]["values"] = [float(round(x, 1)) for x in df_new_france["incid_dc"].rolling(window=7).mean().fillna(0).values[-n:]]
    dict_json["dc"]["dates"] = list(df_france["jour"].values[-n:])
    
    ## Cas
    struct = {"date": "", "values": []}
    dict_json["cas"] = struct
    cas_rolling = df_incid_fra["P"].rolling(window=7, center=False).mean().dropna()
    
    dict_json["cas"]["values"] = [float(round(x, 1)) for x in cas_rolling.values[-n:]]
    dict_json["cas"]["dates"] = list(df_incid_fra.loc[cas_rolling.index.values[-n:], "jour"])
    
    ## Cas date publication
    struct = {"date": "", "values": []}
    dict_json["cas_spf"] = struct
    #cas_rolling = df_vue_ensemble["total_cas_confirmes"].diff().rolling(window=7, center=False).mean().fillna(0)
    cas_rolling = df_opendata_indicateurs["conf_j1"].rolling(window=7, center=False).mean().fillna(0)

    dict_json["cas_spf"]["values"] = [float(round(x, 1)) for x in cas_rolling.values[-n:]]
    dict_json["cas_spf"]["dates"] = list(df_opendata_indicateurs.loc[cas_rolling.index.values[-n:], "date"])
    
    with open(PATH_STATS + 'objectif_deconfinement.json', 'w') as outfile:
        json.dump(dict_json, outfile)
        
objectif_deconfinement()